In [62]:
import pandas as pd

In [63]:
covid = pd.read_hdf('./Covid_data/covid_df.hdf')
pop = pd.read_hdf('./Covid_data/population_df.hdf')
covid_per_cap = pd.read_hdf('./Covid_data/covid_per_cap_df.hdf')
ca_sent = pd.read_hdf('./data/processed/ca_sentiment_mean.hdf')

In [76]:
covid_per_cap_ca = covid_per_cap[pd.datetime(2020,3,19):pd.datetime(2020,8,4)]